<a href="https://colab.research.google.com/github/peterbabulik/Quantum-Native-Entropy-Engine/blob/main/SuperdeterminsmTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import math
from collections import defaultdict

class DeterminismTester:
    def __init__(self, bitstring):
        """
        Expects a string of bits ('100101...') or a list/array of 0s and 1s.
        """
        if isinstance(bitstring, str):
            # Parse string character by character into an array of integers
            parsed_list = [int(bit) for bit in bitstring]
            self.bits = np.array(parsed_list, dtype=np.uint8)
        else:
            self.bits = np.array(bitstring, dtype=np.uint8)
        self.n = len(self.bits)

    def shannon_entropy(self):
        """1. Basic Entropy: Is there a 0 or 1 bias?"""
        if self.n == 0: return 0.0
        p1 = float(np.sum(self.bits)) / self.n
        p0 = 1.0 - p1
        if p1 == 0 or p0 == 0: return 0.0
        return - (p1 * math.log2(p1) + p0 * math.log2(p0))

    def lempel_ziv_complexity(self):
        """
        2. Lempel-Ziv (LZ76) Complexity.
        Approximates Kolmogorov Complexity. True randomness maximizes this.
        """
        if self.n == 0: return 0, 0.0
        s = "".join(map(str, self.bits.tolist()))
        i = 0
        k = 1
        complexity = 1

        # Implement Lempel-Ziv algorithm
        dictionary = {s[0]}
        substring = s[0]
        for bit in s[1:]:
            if substring + bit in dictionary:
                substring += bit
            else:
                dictionary.add(substring + bit)
                complexity += 1
                substring = bit

        # Normalize against expected random complexity: N / log2(N)
        expected_random = self.n / math.log2(self.n) if self.n > 2 else 1.0
        normalized_lz = complexity / expected_random
        return complexity, normalized_lz

    def autocorrelation(self, max_lag=5):
        """
        3. Autocorrelation.
        Checks if the sequence repeats a deterministic pattern every 'k' steps.
        """
        mean = np.mean(self.bits)
        var = np.var(self.bits)

        if var == 0:
            # If variance is zero, all bits are the same, so autocorrelation is undefined or 1
            return [1.0] * max_lag # Return 1 for perfect correlation

        correlations =[]
        for lag in range(1, max_lag + 1):
            if lag >= self.n: break
            # Calculate covariance for shifted arrays
            shifted_original = self.bits[:-lag]
            shifted_lagged = self.bits[lag:]

            cov = np.mean((shifted_original - mean) * (shifted_lagged - mean))
            correlations.append(cov / var)
        return correlations

    def markov_ngram_predictor(self, n_gram_size=4):
        """
        4. State Machine Predictor.
        Finds if a sequence of N bits deterministically dictates the next bit.
        """
        if self.n <= n_gram_size: return 0.0

        # Train: history tracks counts of following a state
        # using a function that returns as default
        def default_counts():
            return {'0': 0, '1': 0}

        history = defaultdict(default_counts)
        for i in range(self.n - n_gram_size):
            state_slice = self.bits[i : i + n_gram_size]
            state = tuple(state_slice)
            next_bit = str(self.bits[i + n_gram_size])
            history[state][next_bit] += 1

        # Predict: test the trained model on the same sequence
        correct = 0
        total_predictions = 0
        for i in range(self.n - n_gram_size):
            state_slice = self.bits[i : i + n_gram_size]
            state = tuple(state_slice)
            actual_next_bit = self.bits[i + n_gram_size]

            counts = history[state]
            if counts['0'] > counts['1']:
                prediction = 0
            elif counts['1'] > counts['0']:
                prediction = 1
            else:
                prediction = np.random.randint(0, 2) # Guess if tied

            if prediction == actual_next_bit:
                correct += 1
            total_predictions += 1

        if total_predictions == 0:
            return 0.0
        return correct / float(total_predictions)

    def binary_matrix_rank(self, M=8, Q=8):
        """
        5. Matrix Rank Test.
        Highly sensitive to linear deterministic equations.
        """
        if self.n < M * Q:
            return None # Not enough bits

        num_blocks = self.n // (M * Q)
        full_rank_count = 0

        for i in range(num_blocks):
            start_idx = i * M * Q
            end_idx = start_idx + (M * Q)
            block_bits = self.bits[start_idx:end_idx]
            block = block_bits.reshape((M, Q))
            rank = np.linalg.matrix_rank(block)
            if rank == min(M, Q):
                full_rank_count += 1

        return full_rank_count / float(num_blocks)

    def run_all_tests(self):
        print(f"--- Running Determinism Tests on sequence of {self.n} bits ---")

        ent = self.shannon_entropy()
        print(f"1. Shannon Entropy: {ent:.4f} (Ideal random = 1.0)")

        lz_comp, lz_norm = self.lempel_ziv_complexity()
        print(f"2. Lempel-Ziv Complexity: {lz_comp} (Normalized: {lz_norm:.4f} - Ideal random ~ 1.0)")

        autocorr = self.autocorrelation(max_lag=5)
        formatted_ac = "[" + ", ".join(f"{val:.4f}" for val in autocorr) + "]" if autocorr else "N/A"
        print(f"3. Autocorrelation (Lags 1-5): {formatted_ac} (Ideal random ~ 0.0)")

        # Example n_gram_sizes for the predictor
        for n_gram in [2, 3, 4]:
            pred_acc = self.markov_ngram_predictor(n_gram_size=n_gram)
            print(f"4. N-Gram Predictor (N={n_gram}): Accuracy {pred_acc:.4f} (Ideal random = 0.5000)")

        rank_ratio = self.binary_matrix_rank(M=8, Q=8)
        if rank_ratio is not None:
            print(f"5. Matrix Full-Rank Ratio: {rank_ratio:.4f} (Expected random ~ 0.2888)")
        else:
            print("5. Matrix Full-Rank Ratio: NOT ENOUGH BITS (Need >= 64)")
        print("-" * 60)


if __name__ == "__main__":
    # 1. Simulating 10,000 bits using NumPy (The Deterministic Simulation)
    np.random.seed(42)
    choices = [0, 1]
    probabilities = [0.5, 0.5]
    simulated_bits = np.random.choice(choices, size=10000, p=probabilities)

    # 2. Mocking QPU bits (Using your exact string from Code 2 output)
    qpu_string = "1001011010000110001101101010011000010000111100011111111010000111"

    # We repeat the QPU string just to have enough length to see how the math spots patterns
    # (In a real test, you should query 10,000 fresh bits from IBM!)
    qpu_bits_mocked = qpu_string * 156

    print("\n NumPy Simulation (Turing Machine / PRNG)")
    tester_sim = DeterminismTester(simulated_bits)
    tester_sim.run_all_tests()

    print("\n Mocked QPU Repeating String (Detecting Deterministic Loops)")
    tester_qpu = DeterminismTester(qpu_bits_mocked)
    tester_qpu.run_all_tests()



 NumPy Simulation (Turing Machine / PRNG)
--- Running Determinism Tests on sequence of 10000 bits ---
1. Shannon Entropy: 0.9998 (Ideal random = 1.0)
2. Lempel-Ziv Complexity: 1304 (Normalized: 1.7327 - Ideal random ~ 1.0)
3. Autocorrelation (Lags 1-5): [-0.0087, -0.0106, -0.0247, -0.0104, -0.0019] (Ideal random ~ 0.0)
4. N-Gram Predictor (N=2): Accuracy 0.5079 (Ideal random = 0.5000)
4. N-Gram Predictor (N=3): Accuracy 0.5127 (Ideal random = 0.5000)
4. N-Gram Predictor (N=4): Accuracy 0.5189 (Ideal random = 0.5000)
5. Matrix Full-Rank Ratio: 0.5897 (Expected random ~ 0.2888)
------------------------------------------------------------

 Mocked QPU Repeating String (Detecting Deterministic Loops)
--- Running Determinism Tests on sequence of 9984 bits ---
1. Shannon Entropy: 1.0000 (Ideal random = 1.0)
2. Lempel-Ziv Complexity: 874 (Normalized: 1.1630 - Ideal random ~ 1.0)
3. Autocorrelation (Lags 1-5): [0.1249, 0.0000, -0.0624, -0.3124, 0.1252] (Ideal random ~ 0.0)
4. N-Gram Predictor

In [3]:
!pip install qiskit qiskit_aer qiskit_ibm_runtime -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 83.7 MB/s eta 0:00:00


In [17]:
import numpy as np
import math
from collections import defaultdict

# Qiskit imports
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import AerSimulator

try:
    from qiskit_ibm_runtime import QiskitRuntimeService
    from qiskit_ibm_runtime import SamplerV2 as Sampler
    from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
    qiskit_runtime_available = True
except ImportError:
    qiskit_runtime_available = False

# ==============================================================================
# 1. THE ALGORITHMIC DETERMINISM TESTER (Bracket-Free UI Bug Bypass)
# ==============================================================================
class DeterminismTester:
    def __init__(self, bit_array, name="Stream"):
        self.bits = np.array(list(bit_array), dtype=np.uint8)
        self.n = len(self.bits)
        self.name = name

    def shannon_entropy(self):
        if self.n == 0: return 0.0
        p1 = float(np.sum(self.bits)) / self.n
        p0 = 1.0 - p1
        if p1 <= 0 or p0 <= 0: return 0.0
        return - (p1 * math.log2(p1) + p0 * math.log2(p0))

    def lempel_ziv_complexity(self):
        if self.n == 0: return 0, 0.0
        s = "".join(map(str, self.bits.tolist()))
        i = 0
        k = 1
        complexity = 1
        while True:
            if i + k > self.n:
                complexity += 1
                break
            # Using slice() to avoid square brackets
            current_substring = s.__getitem__(slice(i, i + k))
            past_substring = s.__getitem__(slice(0, i + k - 1))

            if current_substring in past_substring:
                k += 1
            else:
                complexity += 1
                i += k
                k = 1
        expected_random = self.n / math.log2(self.n) if self.n > 2 else 1.0
        return complexity, complexity / expected_random

    def markov_ngram_predictor(self, n_gram_size=4):
        if self.n <= n_gram_size: return 0.0

        def default_counts():
            return {0: 0, 1: 0}

        history = defaultdict(default_counts)

        # Train
        for i in range(self.n - n_gram_size):
            state_slice = self.bits.__getitem__(slice(i, i + n_gram_size))
            state = tuple(state_slice)
            next_bit = self.bits.__getitem__(i + n_gram_size)

            current_dict = history.get(state)
            if current_dict is None:
                current_dict = default_counts()

            current_val = current_dict.get(next_bit)
            if current_val is None:
                current_val = 0
            current_dict.__setitem__(next_bit, current_val + 1)
            history.__setitem__(state, current_dict)

        # Predict
        correct = 0
        total_predictions = 0
        for i in range(self.n - n_gram_size):
            state_slice = self.bits.__getitem__(slice(i, i + n_gram_size))
            state = tuple(state_slice)
            actual_next_bit = self.bits.__getitem__(i + n_gram_size)

            counts = history.get(state)
            if counts is None:
                counts = default_counts()

            count_0 = counts.get(0)
            count_1 = counts.get(1)
            if count_0 is None:
                count_0 = 0
            if count_1 is None:
                count_1 = 0

            if count_0 > count_1:
                prediction = 0
            elif count_1 > count_0:
                prediction = 1
            else:
                prediction = np.random.randint(0, 2)

            if prediction == actual_next_bit:
                correct += 1
            total_predictions += 1

        return correct / float(total_predictions)

    def binary_matrix_rank(self, M=8, Q=8):
        if self.n < M * Q: return None
        matrices = self.n // (M * Q)
        full_rank_count = 0
        for i in range(matrices):
            start_idx = i * M * Q
            end_idx = start_idx + (M * Q)

            slice_obj = slice(start_idx, end_idx)
            block_1d = self.bits.__getitem__(slice_obj)
            block = block_1d.reshape((M, Q))

            if np.linalg.matrix_rank(block) == min(M, Q):
                full_rank_count += 1
        return full_rank_count / float(matrices)

    def run_all_tests(self):
        print("\n --- Running Tests on " + str(self.n) + " bits ---")
        ent = self.shannon_entropy()
        lz_comp, lz_norm = self.lempel_ziv_complexity()
        pred_acc = self.markov_ngram_predictor(n_gram_size=4)
        rank_ratio = self.binary_matrix_rank(M=8, Q=8)

        print("1. Shannon Entropy       : {:.4f}  (Ideal: 1.0000)".format(ent))
        print("2. Lempel-Ziv Complexity : {}      (Norm: {:.4f}, Ideal: ~1.000)".format(lz_comp, lz_norm))
        print("3. N-Gram Predictor (N=4): Accuracy {:.4f} (Ideal: 0.5000)".format(pred_acc))
        if rank_ratio is not None:
            print("4. Matrix Full-Rank Ratio: {:.4f}  (Ideal: ~0.2888)".format(rank_ratio))
        print("-" * 60)

# ==============================================================================
# 2. GENERATE DETERMINISTIC TURING MACHINE BITS
# ==============================================================================
print("\nGenerating 10,048 bits via Deterministic PRNG (NumPy)...")
np.random.seed(42) # The "Big Bang" seed for our Turing Machine
choices = tuple((0, 1))
probabilities = tuple((0.5, 0.5))
simulated_bits = np.random.choice(choices, size=10048, p=probabilities)

# ==============================================================================
# 3. GENERATE PHYSICAL QUANTUM BITS (IBM QPU)
# ==============================================================================
IBM_QUANTUM_TOKEN = 'your api key here'  # <--- INSERT KEY HERE

qpu_bits = list()

if qiskit_runtime_available and IBM_QUANTUM_TOKEN != 'Your api key here':
    print("\nConnecting to IBM Quantum hardware...")
    service = QiskitRuntimeService(channel='ibm_quantum_platform', token=IBM_QUANTUM_TOKEN)
    backend = service.backend('ibm_torino')
    print("Selected QPU: " + str(backend.name))

    NUM_RANDOM_BITS = 64
    SHOTS = 157

    # Check backend configuration
    backend_config = backend.configuration()
    backend_qubits = backend_config.n_qubits
    print("Backend has " + str(backend_qubits) + " qubits available")

    # Create circuit with explicit registers
    qr = QuantumRegister(NUM_RANDOM_BITS, 'q')
    cr = ClassicalRegister(NUM_RANDOM_BITS, 'meas')
    qc = QuantumCircuit(qr, cr)

    for i in range(NUM_RANDOM_BITS):
        qc.h(qr[i])

    qc.measure(qr, cr)

    pm = generate_preset_pass_manager(backend=backend, optimization_level=0)
    isa_circuit = pm.run(qc)

    # Check how many qubits are in the transpiled circuit
    print("Original circuit qubits: " + str(qc.num_qubits))
    print("Transpiled circuit qubits: " + str(isa_circuit.num_qubits))
    print("Transpiled circuit clbits: " + str(isa_circuit.num_clbits))
    print("Transpiled circuit operations: " + str(isa_circuit.count_ops()))

    print("Executing " + str(SHOTS) + " shots on " + str(backend.name) + " to generate 10,048 bits...")
    sampler = Sampler(mode=backend)

    job_list = list(( tuple((isa_circuit,)), ))
    job = sampler.run(job_list, shots=SHOTS)

    print("Job ID: " + str(job.job_id()) + " submitted. Waiting for physical execution...")
    result = job.result()

    pub_result = result.__getitem__(0)

    # Access the classical register data
    data_attrs = dir(pub_result.data)
    print("Available data attributes: " + str([attr for attr in data_attrs if not attr.startswith('_')]))

    # Find the classical register name
    cr_name = None
    for attr in data_attrs:
        if not attr.startswith('_') and attr not in ['items', 'keys', 'values', 'ndim', 'shape', 'size']:
            cr_name = attr
            break

    if cr_name:
        bit_array_obj = getattr(pub_result.data, cr_name)
        print("Using register: " + str(cr_name))
        print("Bit array shape: " + str(bit_array_obj.array.shape))

        # The bit array stores bits packed in bytes (8 bits per byte)
        # Shape (shots, num_bytes) where num_bytes = num_bits / 8
        raw_matrix = np.asarray(bit_array_obj.array, dtype=np.uint8)

        # Unpack bits from each byte - each row is one shot
        # We need to unpack each byte into 8 bits
        qpu_bits = []
        for shot_idx in range(raw_matrix.shape[0]):
            for byte_val in raw_matrix[shot_idx]:
                # Extract bits from MSB to LSB
                for bit_pos in range(7, -1, -1):
                    bit = (byte_val >> bit_pos) & 1
                    qpu_bits.append(bit)
    else:
        print("Could not find classical register data")

    # Check if we got the expected number of bits
    expected_bits = NUM_RANDOM_BITS * SHOTS
    print("Expected bits: " + str(expected_bits) + ", Extracted: " + str(len(qpu_bits)))

    # If we got fewer bits, we need more shots to get 10,000+ bits
    if len(qpu_bits) < 10000:
        print("WARNING: Got fewer bits than expected. The transpiler may have reduced the circuit.")

    print("Quantum execution complete. Chronological bitstream extracted.")

else:
    print("\nSkipping QPU execution (Missing API Key). Using AerSimulator fallback.")
    backend = AerSimulator()
    qc = QuantumCircuit(64, 64)
    for i in range(64):
        qc.h(i)
    for i in range(64):
        qc.measure(i, i)

    job = backend.run(qc, shots=157, memory=True)
    memory = job.result().get_memory()
    qpu_bits = list()
    for string in memory:
        for char in string:
            qpu_bits.append(int(char))

# ==============================================================================
# 4. THE ULTIMATE COMPARISON
# ==============================================================================
print("\n" + "="*60)
print("             THE SUPERDETERMINISM VERDICT")
print("="*60)

tester_sim = DeterminismTester(simulated_bits, name="Turing Machine PRNG")
tester_sim.run_all_tests()

if len(qpu_bits) > 0:
    tester_qpu = DeterminismTester(qpu_bits, name="IBM Quantum Hardware")
    tester_qpu.run_all_tests()

qiskit_runtime_service._discover_account:WARNING:2026-02-20 18:57:06,118: Loading account with the given token. A saved account will not be used.



Generating 10,048 bits via Deterministic PRNG (NumPy)...

Connecting to IBM Quantum hardware...


qiskit_runtime_service.__init__:WARNING:2026-02-20 18:57:09,121: Instance was not set at service instantiation. Free and trial plan instances will be prioritized. Based on the following filters: (tags: None, region: us-east, eu-de), and available plans: (open), the available account instances are: abcd. If you need a specific instance set it explicitly either by using a saved account with a saved default instance or passing it in directly to QiskitRuntimeService().
qiskit_runtime_service.backends:WARNING:2026-02-20 18:57:09,125: Using instance: abcd, plan: open


Selected QPU: ibm_torino
Backend has 133 qubits available
Original circuit qubits: 64
Transpiled circuit qubits: 133
Transpiled circuit clbits: 64
Transpiled circuit operations: OrderedDict({'rz': 128, 'sx': 64, 'measure': 64})
Executing 157 shots on ibm_torino to generate 10,048 bits...
Job ID: d6cat1ng4t5c7385b2mg submitted. Waiting for physical execution...
Available data attributes: ['items', 'keys', 'meas', 'ndim', 'shape', 'size', 'values']
Using register: meas
Bit array shape: (157, 8)
Expected bits: 10048, Extracted: 10048
Quantum execution complete. Chronological bitstream extracted.

             THE SUPERDETERMINISM VERDICT

 --- Running Tests on 10048 bits ---
1. Shannon Entropy       : 0.9998  (Ideal: 1.0000)
2. Lempel-Ziv Complexity : 784      (Norm: 1.0373, Ideal: ~1.000)
3. N-Gram Predictor (N=4): Accuracy 0.5188 (Ideal: 0.5000)
4. Matrix Full-Rank Ratio: 0.5924  (Ideal: ~0.2888)
------------------------------------------------------------

 --- Running Tests on 10048 b